# Application Example for EvoBandits

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
from application_example import (
    genetic_algorithm,
    TSP_CITIES,
    TSP_OPT_COST,
    TSP_OPT_TOUR
)

In [ ]:
SEED = 42
RNG = np.random.default_rng(SEED)

## 1. Application Example

A genetic algorithm, which solves a fixed instance of the Traveling Salesman Problem (TSP), is applied as example for a stochastic optimization problem. 

The known optimal tour of this 100-city TSP can be used as reference for the evaluation of optimization results.

In [ ]:
tour_path = TSP_CITIES[TSP_OPT_TOUR]
tour_path = np.vstack([tour_path, tour_path[0]])
fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(tour_path[:, 0], tour_path[:, 1], "-", zorder=1)

# Plot cities
ax.scatter(TSP_CITIES[:, 0], TSP_CITIES[:, 1], c="black", zorder=2)

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.grid(True)
ax.axis("equal")

plt.savefig(Path("_plots/01_tsp_opt.pdf"))
plt.show()

In [ ]:
print(f"Cost of the best tour:\t{TSP_OPT_COST}")

## 2. Optimization with EvoBandits

### 2.1 Optimization

The configuration of the genetic algorithm will be optimized using EvoBandits.

Parameter Space for the optimization:

In [ ]:
from evobandits import IntParam, CategoricalParam, FloatParam

params = {
    "pop_size": IntParam(low=50, high=250, size=1),
    "generations": CategoricalParam(choices=[100, 200, 300, 400, 500]),
    "elite_split": FloatParam(low=0.0, high=0.2, n_steps=21), 
    "tournament_split": FloatParam(low=0.0, high=0.1, n_steps=11),
    "mutation_rate": FloatParam(low=0.0, high=1.0, n_steps=101), 
    "crossover_rate": FloatParam(low=0.0, high=1.0, n_steps=101), 
}

Algorithm Configuration:

In [ ]:
from evobandits import GMAB

gmab_instance = GMAB(mutation_span=0.2)

The optimization requires wrapping the genetic algorithm, so that only the objective value (best_cost) is returned as single objective for the optimizer:

In [ ]:
from evobandits import Study

def objective(seed: int, **params: dict):
    """Seeded, single-objective function to simulate the GA."""
    best_cost, _ = genetic_algorithm(seed=seed, **params)
    return best_cost

study = Study(algorithm=gmab_instance, seed=SEED)
study.optimize(objective, params, n_trials=1000, n_runs=3)

### 2.2 Study Output

Display raw results:

In [ ]:
study.results

Aggregate Results:

In [ ]:
print(f"Configuration with best result:\t{study.best_params}")
print(f"Best cost found be evobandits:\t{study.best_value}")
print(f"Mean of best cost across runs:\t{study.mean_value}")

### 2.3 Spread of results from the best configuration

In [ ]:
results = []
for _ in tqdm(range(1000), desc="Collecting Samples"):
    seed = RNG.integers(0, 2**32 - 1, dtype=int)
    best_cost, _ = genetic_algorithm(seed=seed, **study.best_params)
    results.append(best_cost)

json.dump(results, open(Path("_data/02_ga_results_spread.json"), 'w'))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4), gridspec_kw={"width_ratios": [1, 2]})

# Boxplot on left side
axes[0].grid(False)
axes[0].boxplot(results)
axes[0].set_ylabel("Total distance")

# Histplot in 
axes[1].grid()
axes[1].hist(results, bins=20, alpha=0.75)
axes[1].set_xlabel("Total distance")
axes[1].set_ylabel("Frequency")

plt.tight_layout(rect=(0, 0, 1, 0.96))
plt.savefig(Path("_plots/02_ga_results_spread.pdf"))
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
plt.grid()

# Scatter the results
idx = np.arange(1, len(results) + 1)
plt.scatter(idx, results, label="Results", s=10)

# Plot the running means
means = np.cumsum(results) / idx
plt.plot(idx, means, label="Running mean", color="#dd8452")

plt.tight_layout(rect=(0, 0, 1, 0.96))
plt.savefig(Path("_plots/03_ga_running_means.pdf"))
plt.show()